# 🚚 Taller POO: Logística de "Los Pollos Hermanos"

**Rol:** Arquitecto de Software Senior.
**Contexto:** Gustavo Fring ha expandido sus operaciones. Necesitamos un sistema robusto para gestionar la flota de camiones, decodificar mensajes encriptados y registrar cada movimiento.

**Estándares Técnicos (IB Computer Science):**
* **B3.1.5 Encapsulamiento:** Atributos privados.
* **B3.1.3 Variables Estáticas:** Control global.
* **B2.1.3 Manejo de Excepciones:** Robustez con try/except.
* **B2.5.1 File I/O:** Persistencia en UTF-8.

## 1. El Sistema de Rastreo (Static & File I/O)

Usamos `@staticmethod` para herramientas que no requieren crear un objeto, como el sistema de registro (Logging).

In [6]:
import datetime
import os

class SistemaLogistica:
    ARCHIVO_LOG: str = "log_pollos.txt"

    @staticmethod
    def registrar_evento(mensaje: str) -> None:
        hora_actual: str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        linea: str = f"[{hora_actual}] {mensaje}\n"
        
        try:
            # Forzamos encoding utf-8 para tildes en el archivo de texto
            with open(SistemaLogistica.ARCHIVO_LOG, "a", encoding="utf-8") as archivo:
                archivo.write(linea)
            print(f"📑 Log registrado: {mensaje}")
        except IOError as e:
            print(f"❌ ERROR DE E/S: {e}")

    @staticmethod
    def decodificar_codigo(codigo_encriptado: str) -> list[str]:
        if "-" not in codigo_encriptado:
            return []
        return codigo_encriptado.split("-")

## 2. La Clase Base: El Camión (Encapsulamiento)

Implementamos atributos privados (`__`) y métodos de acceso (Getters).

In [7]:
class Camion:
    cantidad_flota: int = 0
    
    def __init__(self, matricula: str, capacidad_maxima: int) -> None:
        self.matricula: str = matricula
        self.capacidad_maxima: int = capacidad_maxima
        
        # Encapsulamiento B3.1.5
        self.__carga_actual: int = 0
        self.__inventario: list[str] = [] 
        
        Camion.cantidad_flota += 1
        SistemaLogistica.registrar_evento(f"Nuevo camión: {self.matricula}")

    def get_carga_actual(self) -> int:
        return self.__carga_actual

    def cargar_producto(self, codigo_envio: str) -> bool:
        datos: list[str] = SistemaLogistica.decodificar_codigo(codigo_envio)
        
        if len(datos) != 3:
            return False
            
        producto: str = datos[0]
        try:
            peso: int = int(datos[2]) 
        except ValueError:
            return False
        
        if (self.__carga_actual + peso) <= self.capacidad_maxima:
            self.__carga_actual += peso
            self.__inventario.append(producto)
            SistemaLogistica.registrar_evento(f"Cargado {peso}kg en {self.matricula}")
            return True
        return False

## 3. Herencia y Polimorfismo

Especialización del camión para productos fríos.

In [3]:
class CamionRefrigerado(Camion):
    def __init__(self, matricula: str, capacidad: int, temperatura: float) -> None:
        super().__init__(matricula, capacidad)
        self.temperatura: float = temperatura
    
    def cargar_producto(self, codigo_envio: str) -> bool:
        if self.temperatura > 4.0:
            SistemaLogistica.registrar_evento(f"ALERTA: {self.matricula} caliente ({self.temperatura}°C)")
            return False
        return super().cargar_producto(codigo_envio)

## 4. Arrays 2D: Mapa de Coordenadas

Navegación por la grilla de Albuquerque.

In [4]:
class MapaAlbuquerque:
    def __init__(self) -> None:
        self.grilla: list[list[str]] = [
            [".", ".", "A"],
            [".", "L", "."],
            ["R", ".", "."]
        ]

    def buscar_ubicacion(self, codigo: str) -> tuple[int, int] | None:
        for i in range(len(self.grilla)):
            for j in range(len(self.grilla[i])):
                if self.grilla[i][j] == codigo:
                    return (i, j)
        return None

## 5. Auditoría Final

Lectura del archivo generado en la sesión.

In [5]:
print("\n--- 🕵️ AUDITORÍA (Lectura de Log) ---")
try:
    with open("log_pollos.txt", "r", encoding="utf-8") as f:
        for linea in f.readlines()[-3:]:
            print(f"  > {linea.strip()}")
except FileNotFoundError:
    print("Archivo no encontrado.")


--- 🕵️ AUDITORÍA (Lectura de Log) ---
